In [2]:
import json

file_name = 'Books_small.json'

with open(file_name) as f:
    for line in f:
        print(line)
        print(line['reviewText'])
        
        break

{"reviewerID": "A1E5ZR1Z4OQJG", "asin": "1495329321", "reviewerName": "Pure Jonel \"Pure Jonel\"", "helpful": [0, 0], "reviewText": "Da Silva takes the divine by storm with this unique new novel.  She develops a world unlike any others while keeping it firmly in the real world.  This is a very well written and entertaining novel.  I was quite impressed and intrigued by the way that this solid storyline was developed, bringing the readers right into the world of the story.  I was engaged throughout and definitely enjoyed my time spent reading it.I loved the character development in this novel.  Da Silva creates a cast of high school students who actually act like high school students.  I really appreciated the fact that none of them were thrown into situations far beyond their years, nor did they deal with events as if they had decades of life experience under their belts.  It was very refreshing and added to the realism and impact of the novel.  The friendships between the characters i

TypeError: string indices must be integers

In [4]:
with open(file_name) as f:
    for line in f:
        review = json.loads(line) #Reading it as a dictionary this time
        print(review['reviewText'])
        print(review['overall'])
        
        break

Da Silva takes the divine by storm with this unique new novel.  She develops a world unlike any others while keeping it firmly in the real world.  This is a very well written and entertaining novel.  I was quite impressed and intrigued by the way that this solid storyline was developed, bringing the readers right into the world of the story.  I was engaged throughout and definitely enjoyed my time spent reading it.I loved the character development in this novel.  Da Silva creates a cast of high school students who actually act like high school students.  I really appreciated the fact that none of them were thrown into situations far beyond their years, nor did they deal with events as if they had decades of life experience under their belts.  It was very refreshing and added to the realism and impact of the novel.  The friendships between the characters in this novel were also truly touching.Overall, this novel was fantastic.  I can&#8217;t wait to read more and to find out what happen

In [13]:
class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: #Score of 4 or 5
            return Sentiment.POSITIVE

In [14]:
reviews = []

with open(file_name) as f:
    for line in f:
        review = json.loads(line) # reading it as a dictionary this time
        reviews.append(Review(review['reviewText'], review['overall']))

In [17]:
reviews[5].sentiment

'POSITIVE'

### Prep Data

In [18]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

In [20]:
len(training)

670

#### Let's pass our BAGS OF WORDS into the vectorizer 

In [21]:
train_x = [x.text for x in training]
train_y = [x.sentiment for x in training]

test_x = [x.text for x in test]
test_y = [x.sentiment for x in test]

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x) #do not want to fit it again since this is our test data 

In [23]:
print(train_x[0])
print(train_x_vectors[0].toarray())

Vivid characters and descriptions. The author has created a tale that grabs your attention and I couldn't put it down.
[[0 0 0 ... 0 0 0]]


### Classification

#### Linear SVM

In [25]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

SVC(kernel='linear')

In [27]:
#predict

test_x[0]

clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

#### Decision Tree

In [29]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(train_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

#### Naive Bayes

In [30]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = DecisionTreeClassifier()
clf_gnb.fit(train_x_vectors, train_y)

clf_gnb.predict(train_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

#### Logistic Regression

In [31]:
from sklearn.linear_model import LogisticRegression

clf_log = DecisionTreeClassifier()
clf_log.fit(train_x_vectors, train_y)

clf_log.predict(train_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Evaluation

#### Mean Accuracy

In [57]:
print("SVM Score = ", round(clf_svm.score(test_x_vectors, test_y)*100),"%")
print("Decision Tree Score = ", round(clf_dec.score(test_x_vectors, test_y)*100),"%")
print("Naive Bayes Score = ", round(clf_gnb.score(test_x_vectors, test_y)*100),"%")
print("Logistic Regression Score = ", round(clf_log.score(test_x_vectors, test_y)*100),"%")

SVM Score =  82 %
Decision Tree Score =  75 %
Naive Bayes Score =  76 %
Logistic Regression Score =  76 %


#### F1 Score

In [52]:
from sklearn.metrics import f1_score

print("SVM Scores[Positive, Neutral, Negative] = ", f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]))
print("Decision Tree Scores[Positive, Neutral, Negative] = ", f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]))
print("Naive Bayes Scores[Positive, Neutral, Negative] = ", f1_score(test_y, clf_gnb.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]))
print("Logistic Regression Scores[Positive, Neutral, Negative] = ", f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]))

SVM Scores[Positive, Neutral, Negative] =  [0.91319444 0.21052632 0.22222222]
Decision Tree Scores[Positive, Neutral, Negative] =  [0.86379928 0.15625    0.10526316]
Naive Bayes Scores[Positive, Neutral, Negative] =  [0.8712522  0.09677419 0.06451613]
Logistic Regression Scores[Positive, Neutral, Negative] =  [0.86925795 0.16949153 0.05714286]


#### Why are the nuetral and negative scores this bad?

In [55]:
train_y.count(Sentiment.NEGATIVE)

47

 ## Next Notebook to cover 10000 data for training